# A primer to HF data from the NOAA Integrated Ocean Observing System (IOOS)

This interactive notebook is a guide to using the NOAA IOOS, specifically the Central and Northern California Ocean Observing System (COOS). We're looking at HF (high frequency) radar data, which we use for hyper-granular measurementes on the order of 1-5 km, or even smaller (500m).

We will use the <u>[HF-Radar Network](https://hfrnet-tds.ucsd.edu/thredds/ncss/grid/HFR/USWC/1km/hourly/RTV/HFRADAR_US_West_Coast_1km_Resolution_Hourly_RTV_best.ncd/dataset.html)</u> from the Coastal Observing Research and Development Center at UCSD.
.
1. Click on Data.
2. Select Data Access via CORDC THREDDS Server. <u>[THREDDS](https://www.unidata.ucar.edu/software/tds/)</u>, or Thematic Real-time Environmental Distributed Data Services, is a server provided through the National Science Foundation's Unidata program to provide researchers to real-time data. As of the time of this writing, I am concerned that the integrity of the server is in question.

![THREDDS](images/THREDDS.png)

3. Follow the link after Data Access.
4. For our purposes, navigate to HF RADAR, US West Coast/
5. <u>[You should be at this link.](https://hfrnet-tds.ucsd.edu/thredds/HFRADAR_USWC.html)</u> Here, you can select data sets of different temporal frequencies and spatial resolutions. 

![catalog](images/catalog.png)

Note that the 500m resolution is available only in the San Francisco Bay.

6. To access this data, go to NetcdfSubset. At this step, you'll be prompted to select variables and spatially and temporally subset your data. You can also choose between an output format of netcdf or netcdf4.

![variables](images/variables.png)

![subset](images/subset.png)

Some serious infrastructural concerns to raise:

- The IOOS, per a <u>[Science.org](https://www.science.org/content/article/trump-seeks-end-climate-research-premier-u-s-climate-agency)</u> article on April 11 this year, is being targeted by the incumbent administration as part of its gutting of NOAA.
- THREDDS is equally under target by the administration.

# SAVE THIS LINK FOR THE DATA CATALOG: https://hfrnet-tds.ucsd.edu/thredds/catalog.html

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install xmovie -qU

In [3]:
from xmovie import Movie
import xmovie

In [4]:
pip install xesmf
pip install multitaper

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import xarray as xr
import numpy as np
import xrft
import xesmf as xe
import matplotlib.pyplot as plt
import cmocean.cm as cm
from dask.diagnostics import ProgressBar

In [6]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap
import matplotlib.ticker as mticker

In [7]:
depth_mask = xr.open_dataset("depth_mask_smooth.nc")

In [8]:
depth_field = xr.open_dataset("depth_field.nc")

In [9]:
wet_field = xr.open_dataset("wet_field.nc")

In [10]:
# Alaska North Slope HR data URLs

#os.environ["REQUESTS_CA_BUNDLE"] = ""  # tells requests/xarray to skip verification

#alaska_6km_hourly = xr.open_dataset("http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/AKNS/6km/hourly/RTV/HFRADAR_Alaska_-_North_Slope_6km_Resolution_Hourly_RTV_best.ncd", engine="netcdf4")
#alaska_6km_25hrly = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/AKNS/6km/25hr/RTV/HFRADAR_Alaska_-_North_Slope_6km_Resolution_25_Hour_Average_RTV_best.ncd")
#alaska_6km_monthly = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/AKNS/6km/LTA/AKNS-month-LTA-6km.nc")

#alaska_6km_hourly.nbytes / 1e9

# alaska_6km_hourly.to_netcdf(alaska_6km_hourly.nc)
# alaska_6km_25hrly.to_netcdf(alaska_6km_25hrly.nc)
# alaska_6km_monthly.to_netcdf(alaska_6km_monthly.nc)

In [11]:
#raise SystemExit("Notebook stopped here.")

In [12]:
# Gulf of Alaska, Hourly

#gulf_of_alaska_1km_hourly = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/1km/hourly/RTV/HFRADAR_US_Gulf_of_Alaska_1km_Resolution_Hourly_RTV_best.ncd")
#gulf_of_alaska_2km_hourly = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/2km/hourly/RTV/HFRADAR_US_Gulf_of_Alaska_2km_Resolution_Hourly_RTV_best.ncd")
#gulf_of_alaska_6km_hourly = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/6km/hourly/RTV/HFRADAR_US_Gulf_of_Alaska_6km_Resolution_Hourly_RTV_best.ncd")

In [13]:
# Gulf of Alaska, 25 Hour

#gulf_of_alaska_1km_25hr = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/1km/25hr/RTV/HFRADAR_US_Gulf_of_Alaska_1km_Resolution_25_Hour_Average_RTV_best.ncd")
#gulf_of_alaska_2km_25hr = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/2km/25hr/RTV/HFRADAR_US_Gulf_of_Alaska_2km_Resolution_25_Hour_Average_RTV_best.ncd")
#gulf_of_alaska_6km_25hr = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/6km/25hr/RTV/HFRADAR_US_Gulf_of_Alaska_6km_Resolution_25_Hour_Average_RTV_best.ncd")

In [14]:
# Gulf of Alaska, Monthly

#gulf_of_alaska_1km_month = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/1km/LTA/GAK-month-LTA-1km.nc")
#gulf_of_alaska_2km_month = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/2km/LTA/GAK-month-LTA-2km.nc")
#gulf_of_alaska_6km_month = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/6km/LTA/GAK-month-LTA-6km.nc")

In [15]:
# Gulf of Alaska, Yearly

#gulf_of_alaska_1km_yearly = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/1km/LTA/GAK-annual-LTA-1km.nc")
#gulf_of_alaska_2km_yearly = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/2km/LTA/GAK-annual-LTA-2km.nc")
#gulf_of_alaska_6km_yearly = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/GAK/6km/LTA/GAK-annual-LTA-6km.nc")

In [16]:
#ds_test_west = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd")

In [17]:
#ds_east = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USEGC/6km/hourly/RTV/HFRADAR_US_East_and_Gulf_Coast_6km_Resolution_Hourly_RTV_best.ncd")

In [18]:
#ds_alaska = xr.open_dataset("https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/AKNS/6km/hourly/RTV/HFRADAR_Alaska_-_North_Slope_6km_Resolution_Hourly_RTV_best.ncd")

In [19]:
# Replace with your file path
ds_500m = xr.open_dataset("RTV_HFRADAR_US_West_Coast_500m_Resolution_Hourly_RTV_best.nc4")

# View dataset summary
ds_500m

# Access a specific variable (e.g., eastward velocity)
u = ds_500m['u']

# Plot a slice
u.isel(time=0).plot()

# Counts the number of values at each timestep
#u.count(dim=['lat', 'lon']).values

In [20]:
ds_6km = xr.open_dataset("RTV_HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.nc4")

u = ds_6km['u']
v = ds_6km['v']

u.count(dim=['lat', 'lon']).values

array([7718, 7262, 7074, ..., 7681, 7373, 6601], shape=(12341,))

In [21]:
ds_6km['speed'] = np.sqrt(np.square(u) + np.square(v))

In [22]:
ds_6km

<xarray.Dataset> Size: 13GB
Dimensions:   (time: 12341, lat: 367, lon: 234)
Coordinates:
    time_run  (time) datetime64[ns] 99kB ...
  * time      (time) datetime64[ns] 99kB 2024-01-01 ... 2025-05-29T04:00:00
  * lat       (lat) float32 1kB 30.25 30.3 30.36 30.41 ... 49.88 49.94 49.99
  * lon       (lon) float32 936B -130.4 -130.3 -130.2 ... -115.9 -115.9 -115.8
Data variables:
    u         (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    wgs84     int8 1B ...
    v         (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    speed     (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
Attributes: (12/31)
    Conventions:               CF-1.4, ACDD-1.3
    id:                        202505141600siohfruwlsrtvuswc6km
    date_created:              2025-05-14T18:24:43Z
    source:                    surface ocean velocity field from hf-radar
    program:                   Integrated Ocean Observing System (IOOS)
    title:                     Near-Real Time Surface Ocean Velocity, U.S. We...
    ...                        ...
    format_version:            1.1.00
    product_version:           2.0.00
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:HFRADAR_US_West_Coast_6km_Resolutio...
    History:                   Translated to CF-1.0 Conventions by Netcdf-Jav...

# Speed subset: we look at the subsetted coastal area on the HF data

In [23]:
lon_min, lon_max, lat_min, lat_max = -126, -121, 32.5, 45

speed_subset = ds_6km.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
speed_subset

<xarray.Dataset> Size: 3GB
Dimensions:   (time: 12341, lat: 232, lon: 80)
Coordinates:
    time_run  (time) datetime64[ns] 99kB ...
  * time      (time) datetime64[ns] 99kB 2024-01-01 ... 2025-05-29T04:00:00
  * lat       (lat) float32 928B 32.52 32.57 32.62 32.68 ... 44.87 44.92 44.98
  * lon       (lon) float32 320B -126.0 -125.9 -125.9 ... -121.2 -121.1 -121.1
Data variables:
    u         (time, lat, lon) float32 916MB nan nan nan nan ... nan nan nan nan
    wgs84     int8 1B ...
    v         (time, lat, lon) float32 916MB nan nan nan nan ... nan nan nan nan
    speed     (time, lat, lon) float32 916MB nan nan nan nan ... nan nan nan nan
Attributes: (12/31)
    Conventions:               CF-1.4, ACDD-1.3
    id:                        202505141600siohfruwlsrtvuswc6km
    date_created:              2025-05-14T18:24:43Z
    source:                    surface ocean velocity field from hf-radar
    program:                   Integrated Ocean Observing System (IOOS)
    title:                     Near-Real Time Surface Ocean Velocity, U.S. We...
    ...                        ...
    format_version:            1.1.00
    product_version:           2.0.00
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:HFRADAR_US_West_Coast_6km_Resolutio...
    History:                   Translated to CF-1.0 Conventions by Netcdf-Jav...

In [24]:
transparent_lightblue_cmap = ListedColormap([
    (1, 1, 1.0, 0.0),    # clear transparent
    (0.0, 0.0, 1.0, 0.0)   # blue
])

speed_subset.speed.isel(time=0).plot(vmin=-0.5, vmax=0.5)
depth_mask["depth"].plot(
    cmap=transparent_lightblue_cmap, x="XC", y="YC",
    vmin=0, vmax=1,        # forces correct index lookup
    #add_colorbar=False, 
    alpha=0.1)

# interpolate mask to the HF data
# inerpolate depth field to HF data, THEN create the mask from HF data
# picture of residual flow, get bathymetry of different regions (focus on one at a time)
# overlying winds at some point down the line
# get metric of total number of points per time step, make time series graph of that so we can see data density throughout time
# hourly time series field map, and then do running mean over 24 hours (do your own time smoothing), third panel that's difference between the two
# speed: u and v, do running mean on u and v separately

In [25]:
depth_mask["depth"]
#u_subset

<xarray.DataArray 'depth' (j: 835, i: 239)> Size: 2MB
[199565 values with dtype=int64]
Coordinates: (12/19)
  * j        (j) int64 7kB 9711 9712 9713 9714 9715 ... 10542 10543 10544 10545
  * i        (i) int64 2kB 13056 13057 13058 13059 ... 13291 13292 13293 13294
    time     datetime64[ns] 8B ...
    CS       (j, i) float32 798kB ...
    Depth    (j, i) float32 798kB ...
    PHrefC   float32 4B ...
    ...       ...
    hFacC    (j, i) float32 798kB ...
    iter     int64 8B ...
    k        int64 8B ...
    k_l      int64 8B ...
    k_u      int64 8B ...
    rA       (j, i) float32 798kB ...

# An interesting point regarding going from curvilinear to rectilinear grids and vice-versa...

We have a problem: depth_field has dimensions in $j$, $i$ -- a rectilinear grid -- but we need to change to curvilinear grid (lat/lon).

Firstly, "XC" and "YC" in depth_field are the lon and lat coordinates, respectively. Ensure that we have lat/lon coordinates as follows:

In [26]:
depth_field_latlon = depth_field.assign_coords(lat=depth_field.YC, lon=depth_field.XC)
wet_field_latlon = wet_field.assign_coords(lat=wet_field.YC, lon=wet_field.XC)

In [27]:
source_grid = {
    'lon': depth_field_latlon.lon,
    'lat': depth_field_latlon.lat
}

target_grid = {
    'lon': ds_6km.lon, # change this based on the target dataset
    'lat': ds_6km.lat
}

regridder = xe.Regridder(source_grid, target_grid, method='bilinear', extrap_method='nearest_s2d', ignore_degenerate=True)
depth_field_interpolated = regridder(depth_field_latlon)

regridder_nearest = xe.Regridder(source_grid, target_grid, method='nearest_s2d', extrap_method='nearest_s2d', ignore_degenerate=True)
depth_field_interpolated_nearest = regridder(depth_field_latlon)

In [28]:
source_grid = {
    'lon': wet_field_latlon.lon,
    'lat': wet_field_latlon.lat
}

target_grid = {
    'lon': ds_6km.lon,
    'lat': ds_6km.lat
}

regridder = xe.Regridder(source_grid, target_grid, method='bilinear', extrap_method='nearest_s2d', ignore_degenerate=True)
wet_field_interpolated = regridder(wet_field_latlon)

regridder_nearest = xe.Regridder(source_grid, target_grid, method='nearest_s2d', extrap_method='nearest_s2d', ignore_degenerate=True)
wet_field_interpolated_nearest = regridder(wet_field_latlon)

In [29]:
depth_field_interpolated.to_netcdf('depth_field_interpolated.nc')

In [30]:
wet_field_interpolated.to_netcdf('wet_field_interpolated.nc')

In [31]:
depth_field_interpolated = xr.open_dataset('depth_field_interpolated.nc')

In [32]:
depth_field_interpolated.depth.plot()

In [33]:
wet_field_interpolated = xr.open_dataset('wet_field_interpolated.nc')

In [34]:
wet_field_interpolated.wet_mask.plot()
plt.savefig("wet_mask_plot.png", dpi=300, bbox_inches="tight")
plt.close()

# Depth: Bilinear

In [35]:
# Create figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Plot depth as the background
depth_field_interpolated.depth.plot(ax=ax, cmap="Blues", alpha=0.7)  # adjust alpha if needed

ax.set_title("Depth, Interpolated")
#plt.show()
#plt.savefig('depth-bilinear.png')

Text(0.5, 1.0, 'Depth, Interpolated')

# Depth: Nearest

In [36]:
# Create figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Plot depth as the background
depth_field_interpolated_nearest.depth.plot(ax=ax, cmap="Blues", alpha=0.7)  # adjust alpha if needed

ax.set_title("Depth, Interpolated")
#plt.show()
plt.savefig('depth-nearest.png')

In [37]:
# Create figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Plot depth as the background
depth_field_interpolated.depth.plot(ax=ax, cmap="Blues", alpha=0.7)  # adjust alpha if needed

# Overlay u_subset on top with color limits
speed_subset.speed.isel(time=0).plot(ax=ax, vmin=0, vmax=0.5, cmap="RdBu", alpha=0.4)

ax.set_title("Depth + Speed")
#plt.show()
plt.savefig('test.png')


In [38]:
depth_clean = depth_field_interpolated.reset_coords(drop=True)
depth_clean

<xarray.Dataset> Size: 346kB
Dimensions:  (lat: 367, lon: 234)
Coordinates:
  * lat      (lat) float32 1kB 30.25 30.3 30.36 30.41 ... 49.88 49.94 49.99
  * lon      (lon) float32 936B -130.4 -130.3 -130.2 ... -115.9 -115.9 -115.8
Data variables:
    depth    (lat, lon) float32 344kB ...
Attributes:
    regrid_method:  bilinear

In [39]:
wet_clean = wet_field_interpolated.reset_coords(drop=True)
wet_clean

<xarray.Dataset> Size: 88kB
Dimensions:   (lat: 367, lon: 234)
Coordinates:
  * lat       (lat) float32 1kB 30.25 30.3 30.36 30.41 ... 49.88 49.94 49.99
  * lon       (lon) float32 936B -130.4 -130.3 -130.2 ... -115.9 -115.9 -115.8
Data variables:
    wet_mask  (lat, lon) bool 86kB ...
Attributes:
    regrid_method:  bilinear

In [40]:
speed_subset_clean = speed_subset.reset_coords(drop=True)
#ds_6km_slice = ds_6km_clean.isel(time=slice(-500, None))
speed_subset_clean

<xarray.Dataset> Size: 3GB
Dimensions:  (time: 12341, lat: 232, lon: 80)
Coordinates:
  * time     (time) datetime64[ns] 99kB 2024-01-01 ... 2025-05-29T04:00:00
  * lat      (lat) float32 928B 32.52 32.57 32.62 32.68 ... 44.87 44.92 44.98
  * lon      (lon) float32 320B -126.0 -125.9 -125.9 ... -121.2 -121.1 -121.1
Data variables:
    u        (time, lat, lon) float32 916MB nan nan nan nan ... nan nan nan nan
    wgs84    int8 1B ...
    v        (time, lat, lon) float32 916MB nan nan nan nan ... nan nan nan nan
    speed    (time, lat, lon) float32 916MB nan nan nan nan ... nan nan nan nan
Attributes: (12/31)
    Conventions:               CF-1.4, ACDD-1.3
    id:                        202505141600siohfruwlsrtvuswc6km
    date_created:              2025-05-14T18:24:43Z
    source:                    surface ocean velocity field from hf-radar
    program:                   Integrated Ocean Observing System (IOOS)
    title:                     Near-Real Time Surface Ocean Velocity, U.S. We...
    ...                        ...
    format_version:            1.1.00
    product_version:           2.0.00
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:HFRADAR_US_West_Coast_6km_Resolutio...
    History:                   Translated to CF-1.0 Conventions by Netcdf-Jav...

In [41]:
fig, ax = plt.subplots(figsize=(8, 6))  # optional: adjust size
speed_subset_clean.speed.isel(time=0).plot(ax=ax)

plt.savefig('speed-pre-merge.png', bbox_inches='tight', dpi=300)
plt.close(fig)  # prevents figure overlap in loops or scripts

In [42]:
fig, ax = plt.subplots(figsize=(8, 6))  # optional: adjust size
depth_clean.depth.plot(ax=ax)

plt.savefig('depth-pre-merge.png', bbox_inches='tight', dpi=300)
plt.close(fig)  # prevents figure overlap in loops or scripts

# The moment we've all been waiting for: merging the HF and LLC depth data... (the lat/lon coordinates for both xarray objects are the same shape!)

In [43]:
merge_depth_hf = xr.merge([speed_subset_clean, depth_clean])

In [44]:
# merge_depth_hf.to_netcdf("merge_depth_hf.nc")

In [45]:
#merge_depth_hf = xr.open_dataset("merge_depth_hf.nc")
merge_depth_hf

<xarray.Dataset> Size: 13GB
Dimensions:  (time: 12341, lat: 367, lon: 234)
Coordinates:
  * time     (time) datetime64[ns] 99kB 2024-01-01 ... 2025-05-29T04:00:00
  * lat      (lat) float32 1kB 30.25 30.3 30.36 30.41 ... 49.88 49.94 49.99
  * lon      (lon) float32 936B -130.4 -130.3 -130.2 ... -115.9 -115.9 -115.8
Data variables:
    u        (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    wgs84    int8 1B ...
    v        (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    speed    (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    depth    (lat, lon) float32 344kB ...
Attributes: (12/31)
    Conventions:               CF-1.4, ACDD-1.3
    id:                        202505141600siohfruwlsrtvuswc6km
    date_created:              2025-05-14T18:24:43Z
    source:                    surface ocean velocity field from hf-radar
    program:                   Integrated Ocean Observing System (IOOS)
    title:                     Near-Real Time Surface Ocean Velocity, U.S. We...
    ...                        ...
    format_version:            1.1.00
    product_version:           2.0.00
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:HFRADAR_US_West_Coast_6km_Resolutio...
    History:                   Translated to CF-1.0 Conventions by Netcdf-Jav...

In [46]:
fig, ax = plt.subplots(figsize=(8, 6))  # adjust size if needed
merge_depth_hf.isel(time=0).speed.plot(ax=ax)

plt.savefig('speed.png', bbox_inches='tight', dpi=300)
plt.close(fig)

In [47]:
# Create figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Plot depth as the background
merge_depth_hf.depth.plot(ax=ax, cmap="Blues", alpha=0.7)  # adjust alpha if needed

# Overlay u_subset on top with color limits
merge_depth_hf.speed.isel(time=0).plot(ax=ax, vmin=0, vmax=0.5, cmap="RdBu", alpha=0.4)

ax.set_title("Depth + Speed")
#plt.show()
plt.savefig('test1.png')

In [48]:
merge_depth_hf

<xarray.Dataset> Size: 13GB
Dimensions:  (time: 12341, lat: 367, lon: 234)
Coordinates:
  * time     (time) datetime64[ns] 99kB 2024-01-01 ... 2025-05-29T04:00:00
  * lat      (lat) float32 1kB 30.25 30.3 30.36 30.41 ... 49.88 49.94 49.99
  * lon      (lon) float32 936B -130.4 -130.3 -130.2 ... -115.9 -115.9 -115.8
Data variables:
    u        (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    wgs84    int8 1B ...
    v        (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    speed    (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    depth    (lat, lon) float32 344kB ...
Attributes: (12/31)
    Conventions:               CF-1.4, ACDD-1.3
    id:                        202505141600siohfruwlsrtvuswc6km
    date_created:              2025-05-14T18:24:43Z
    source:                    surface ocean velocity field from hf-radar
    program:                   Integrated Ocean Observing System (IOOS)
    title:                     Near-Real Time Surface Ocean Velocity, U.S. We...
    ...                        ...
    format_version:            1.1.00
    product_version:           2.0.00
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:HFRADAR_US_West_Coast_6km_Resolutio...
    History:                   Translated to CF-1.0 Conventions by Netcdf-Jav...

# Building the depth mask:

In [49]:
depth_mask = merge_depth_hf.depth <= 3000

masked_merge_depth_hf = merge_depth_hf.where(depth_mask)

In [50]:
masked_merge_depth_hf

<xarray.Dataset> Size: 13GB
Dimensions:  (time: 12341, lat: 367, lon: 234)
Coordinates:
  * time     (time) datetime64[ns] 99kB 2024-01-01 ... 2025-05-29T04:00:00
  * lat      (lat) float32 1kB 30.25 30.3 30.36 30.41 ... 49.88 49.94 49.99
  * lon      (lon) float32 936B -130.4 -130.3 -130.2 ... -115.9 -115.9 -115.8
Data variables:
    u        (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    wgs84    (lat, lon) float32 344kB nan nan nan nan ... -127.0 -127.0 -127.0
    v        (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    speed    (time, lat, lon) float32 4GB nan nan nan nan ... nan nan nan nan
    depth    (lat, lon) float32 344kB nan nan nan nan nan ... 0.0 0.0 0.0 0.0
Attributes: (12/31)
    Conventions:               CF-1.4, ACDD-1.3
    id:                        202505141600siohfruwlsrtvuswc6km
    date_created:              2025-05-14T18:24:43Z
    source:                    surface ocean velocity field from hf-radar
    program:                   Integrated Ocean Observing System (IOOS)
    title:                     Near-Real Time Surface Ocean Velocity, U.S. We...
    ...                        ...
    format_version:            1.1.00
    product_version:           2.0.00
    cdm_data_type:             GRID
    featureType:               GRID
    location:                  Proto fmrc:HFRADAR_US_West_Coast_6km_Resolutio...
    History:                   Translated to CF-1.0 Conventions by Netcdf-Jav...

In [51]:
def hf_plotter(ds, fig, tt, *args, **kwargs):
    da = ds.isel(time=tt)
    ax = fig.add_subplot(1, 1, 1)

    mesh = ax.pcolormesh(ds.lon, ds.lat, da, cmap='coolwarm', shading='auto', vmin=0, vmax=1)
    ax.set_title( (da.time).dt.strftime("%H, %d %b, 20%y").values.item() )
    fig.colorbar(mesh, ax=ax, label="Speed (m/s)")
    
    fig.suptitle('HF Radar Data at Shallow Depth (<= 3000 m), US West Coast, 6km: Surface Flow Speed (m/s)') # add title as argument of function
    
    fig.tight_layout()
    
    return ax, mesh

In [52]:
def rolling_hf_plotter(ds, fig, tt, *args, **kwargs):
    da = ds.demeaned.isel(time=tt)
    ax = fig.add_subplot(1, 1, 1)

    mesh = ax.pcolormesh(ds.lon, ds.lat, da, cmap='coolwarm', shading='auto', vmin=-0.6, vmax=0.6) # change vmin, vmax depending on if we're using speed or demeaned
    ax.contour(ds.lon, ds.lat, ds.final_mask.astype(int), levels=[0.5], colors='black')
    ax.set_title((da.time).dt.strftime("%H, %d %b, %Y").values.item())
    fig.colorbar(mesh, ax=ax, label="Speed (m/s)")

    # Use the `title` keyword argument, or fall back to a default string
    suptitle = kwargs.get("title", "default title")
    fig.suptitle(suptitle)

    fig.tight_layout()
    return ax, mesh


In [53]:
raise SystemExit("Notebook stopped here.")

SystemExit: Notebook stopped here.

/srv/conda/envs/notebook/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3556: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Making a movie of speed only:

In [ ]:
mov = Movie(ds_6km.speed.chunk({'time':1}).isel(time=slice(-3000, None, 1)), 
            hf_plotter,
            input_check=False, dpi=100, 
            pixelwidth=1600,
            pixelheight=1240)

with ProgressBar():
    mov.save('west6km-3000.mp4',
            progress=True,
            parallel=True,
            overwrite_existing=True,
            #parallel_compute_kwargs=dict(scheduler="processes", num_workers=6)
            )

# 187.51 seconds for 250 frames
# 531.52 seconds for 500 frames
# 17m 31s (1031 seconds) for 750 frames
# 25m 53s (1553 seconds) for 1000 frames
# 95m 58s (5758 seconds) for 2000 frames
# 2hrs 46 min for 57% of 3000 frames
# polynomial time?

# Now for rolling mean of speed:

In [ ]:
rolling_ds_6km = ds_6km.rolling(time=24, center=True).mean()
rolling_u = u.rolling(time=24, center=True).mean()
rolling_v = v.rolling(time=24, center=True).mean()

rolling_ds_6km['speed'] = np.sqrt(np.square(rolling_u) + np.square(rolling_v))

In [ ]:
rolling_ds_6km.speed.isel(time=0).plot()

In [ ]:
rolling_ds_6km

In [ ]:
merge_hf_rolling_daily = xr.merge([rolling_ds_6km, depth_clean, wet_clean])

In [ ]:
rolling_depth_mask = (merge_hf_rolling_daily.depth <= 3000).fillna(False)

merge_hf_rolling_daily['prelim_mask'] = rolling_depth_mask

merge_hf_rolling_daily['final_mask'] = rolling_depth_mask & merge_hf_rolling_daily.wet_mask

merge_hf_rolling_daily['demeaned'] = ds_6km.speed - merge_hf_rolling_daily.speed

#merge_hf_rolling_daily.to_netcdf("merge_hf_rolling_daily.nc")

In [ ]:
#merge_hf_rolling_daily = xr.open_dataset("merge_hf_rolling_daily.nc")

In [ ]:
merge_hf_rolling_daily

In [ ]:
merge_hf_rolling_daily.demeaned.isel(time=0).plot()
plt.savefig('demeaned.png', bbox_inches='tight', dpi=300)
plt.close(fig)

In [ ]:
mov = Movie(merge_hf_rolling_daily.chunk({'time':1}).isel(time=slice(-2000, None, 1)), 
            rolling_hf_plotter,
            input_check=False, dpi=100, 
            pixelwidth=1600,
            pixelheight=1240,
            title="HF Radar Data Anomaly (De-meaned), US West Coast, 6km: Surface Flow Speed (m/s)")

with ProgressBar():
    mov.save('demeaned-rolling-west6km-2000-contour.mp4',
            progress=True,
            parallel=True,
            overwrite_existing=True,
            #parallel_compute_kwargs=dict(scheduler="processes", num_workers=6)
            )

# Some fun with plotting algorithmic complexity for imovie...

In [ ]:
from scipy.optimize import curve_fit

# Example data
x_data = np.array([250, 500, 750, 1000, 2000])
y_data = np.array([187.51, 531.52, 1031, 1553, 5758])

# Define polynomial model (quadratic)
def poly2(x, a, b, c):
    return a * x**2 + b * x + c

# Fit model
popt, pcov = curve_fit(poly2, x_data, y_data)
a, b, c = popt
print(f"Fitted coefficients:\na = {a:.4f}, b = {b:.4f}, c = {c:.4f}")

# Evaluate fitted curve over extended range
x_fit = np.linspace(0, 9000, 300)  # 0 to 9000
y_fit = poly2(x_fit, *popt)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(x_data, y_data, label='Data', color='blue')
ax.plot(x_fit, y_fit, label='Fitted Curve', color='red')
ax.set_xlim(0, 9000)  # Set x-axis range explicitly
ax.set_title('Nonlinear Least Squares Polynomial Fit')
ax.set_xlabel('Number of Frames')
ax.set_ylabel('Seconds')
ax.legend()
ax.grid(True)

plt.savefig('imoviecomplexity.png', bbox_inches='tight', dpi=300)
plt.close(fig)

# In order to make a movie of around 8760 frames, it would take 27 hrs 46 min 40 sec.

# Making a movie of masked HF data (selecting only for speed at shallow >= 3000 m depths):

In [ ]:
if False:
    mov = Movie(masked_merge_depth_hf.speed.chunk({'time':1}).isel(time=slice(-4000, None, 1)), 
                hf_plotter,
                input_check=False, dpi=100, 
                pixelwidth=1600,
                pixelheight=1240)
    
    with ProgressBar():
        mov.save('masked_hf_4000.mp4',
                progress=True,
                parallel=True,
                overwrite_existing=True,
                #parallel_compute_kwargs=dict(scheduler="processes", num_workers=6)
                )

# draw contour line on full field plot to delineate shallow region

# fluid mechanics: chapter 12, turbulence (5ed), reading sections 12.1-12.7

# Creating line plot of number of values each frame of masked_merge_depth_hf has:

In [ ]:
speed_sample_size = []

for i in range(masked_merge_depth_hf.time.size):
    speed_sample_size.append(masked_merge_depth_hf.speed.isel(time=i).isnull().sum().values.item())

In [ ]:
import matplotlib.dates as mdates
from matplotlib.dates import MonthLocator, DateFormatter

# Extract time and values
time_values = masked_merge_depth_hf.time.values
values = speed_sample_size  # if still array(15477), etc.

# Create plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(time_values, values, linestyle='-', color='black', linewidth=0.5)  # No marker, thinner line

# Set ticks every 6 months
#ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

ax.xaxis.set_major_locator(MonthLocator(bymonth=(1, 3, 5, 7, 9, 11)))
ax.xaxis.set_major_formatter(DateFormatter('%b %Y'))

ax.set_xlim(time_values[0], time_values[-1])
ax.set_ylim(14500, )

# Beautify
plt.xticks(rotation=45, ha='right')
plt.xlabel("Date")
plt.ylabel("Non-NaN Speed Points")
plt.title("Speed Sample Size Over Time (US Western Coast Region)")
plt.tight_layout()
plt.grid(True)
plt.savefig('speed_data_density.png')

 Two ways to frame this project:

 1) SWOT intercomparison, only care about HF data after SWOT was released
 2) energetics need long term

for swot intercomparison, need to decide target grid, formulate transform of swot onto target grid (nearest neighbor, binning)

# Plotting radar stations

In [ ]:
stationlist = pd.read_csv("stationlist.csv")

stationlist["presence"] = 1

stationlist = stationlist.rename(columns={"long": "lon"})

stationlist = stationlist.drop_duplicates(subset=["lat", "lon"])

grid = stationlist.pivot(index="lat", columns="lon", values="presence")

stationlist = xr.DataArray(
    data=grid.values,
    coords={"lat": grid.index.values, "lon": grid.columns.values},
    dims=["lat", "lon"]
)

In [ ]:
stationlist = pd.read_csv("stationlist.csv")
stationlist = stationlist.rename(columns={"long": "lon"})

western_stations = stationlist[
    (stationlist["lat"] >= 32.0) & (stationlist["lat"] <= 50.0) &
    (stationlist["lon"] >= -127.0) & (stationlist["lon"] <= -116.0)
]

In [ ]:
plt.figure(figsize=(12, 8))

ax = plt.axes(projection=ccrs.PlateCarree())

# Plot gridded speed data (uses xarray's built-in plotting with Cartopy support)
ds_6km.isel(time=1).speed.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='viridis')

# Overlay radar stations as red dots
ax.scatter(
    western_stations["lon"], western_stations["lat"],
    color='red', s=40, edgecolor='black', linewidth=0.5,
    transform=ccrs.PlateCarree(),
    label='Radar Stations'
)

ax.set_extent([-127, -116, 32, 50], crs=ccrs.PlateCarree())

# Add lat/lon gridlines
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.7, linestyle='--')
gl.top_labels = False     # Hide top x-axis labels
gl.right_labels = False   # Hide right y-axis labels

# Optional: force labels to appear at nice intervals
gl.xlabel_style = {"size": 10}
gl.ylabel_style = {"size": 10}

ax.coastlines()
ax.legend(
    loc='center right',
    bbox_to_anchor=(0.6, 0.05),
    frameon=False
)
ax.set_title("6km Speed and Radar Stations at Time Index 1")

plt.savefig("stationlist.png")

# Plotting data density

In [ ]:
valid_counts = ds_6km.speed.notnull().sum(dim="time")

In [ ]:
total_steps = ds_6km.dims["time"]

In [ ]:
data_density = (valid_counts / total_steps) * 100

In [ ]:
# Define projection and figure
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={"projection": projection})

# Plot data density with masked 0s and custom colormap (optional)
masked_density = data_density.where(data_density > 0)
cmap = plt.get_cmap("viridis").copy()
cmap.set_bad("white")

density_plot = masked_density.plot(
    ax=ax,
    transform=projection,
    cmap=cmap,
    vmin=0,
    vmax=100,
    add_colorbar=True,
    cbar_kwargs={
        "label": "Data Coverage",
        "format": mticker.FuncFormatter(lambda x, _: f"{x:.0f}%")
    }
)

# Plot radar stations
ax.scatter(
    western_stations["lon"], western_stations["lat"],
    color='red', s=40, edgecolor='black', linewidth=0.5,
    transform=projection,
    label='Radar Stations'
)

# Set map extent to California/west coast
ax.set_extent([-127, -116, 32, 50], crs=projection)

# Add coastlines
ax.coastlines()

# Add gridlines and lat/lon labels
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.7, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {"size": 10}
gl.ylabel_style = {"size": 10}

# Place legend outside plot
ax.legend(
    loc='center right',
    bbox_to_anchor=(0.9, 0.5),
    frameon=False
)

# Title and save
ax.set_title("Data Availability (% of Time) at Each Location (1/1/24 - 5/29/25)")
plt.tight_layout()
plt.savefig("datadensity.png", dpi=100)

# Time-mean flow

In [ ]:
normalized_u = ds_6km.u / ds_6km.speed
time_mean_normalized_u = normalized_u.mean(dim="time")[::7, ::7]
normalized_v = ds_6km.v / ds_6km.speed
time_mean_normalized_v = normalized_v.mean(dim="time")[::7, ::7]
time_mean_ds_6km = ds_6km.mean(dim="time")

In [ ]:
# Define projection and figure
projection = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={"projection": projection})

# Plot data density with masked 0s and custom colormap (optional)
time_mean = time_mean_ds_6km.speed.where(~np.isnan(time_mean_ds_6km.speed))
cmap = plt.get_cmap("jet").copy()
cmap.set_bad("white")

# Time-mean plot
time_mean_plot = time_mean.plot(
    ax=ax,
    transform=projection,
    cmap=cmap,
    vmin=0,
    vmax=0.5,
    add_colorbar=True,
    cbar_kwargs={
        "label": "Mean Flow (m s$^{-1}$)",
    }
)

# Create a binary mask for low-density regions (0 < density < 50)
low_density_mask = ((data_density > 0) & (data_density < 50)).astype(int)

# Plot hatching over low-density regions
hatch_contour = ax.contourf(
    data_density['lon'],  # replace with your longitude coordinate name
    data_density['lat'],  # replace with your latitude coordinate name
    low_density_mask,
    levels=[0.5, 1.5],     # target the "1" values in the mask
    colors='none',         # no fill, just hatch
    hatches=['xxx'],
    transform=ccrs.PlateCarree()  # if you're using Cartopy
)

# add arrows for flow directions
ax.quiver(
    time_mean_normalized_u.lon, time_mean_normalized_u.lat,  # grid positions
    time_mean_normalized_u, time_mean_normalized_v,                # vector components
    scale=20            # adjust size of arrows
)

# Set map extent to California/west coast
ax.set_extent([-127, -116, 32, 50], crs=projection)

# Add coastlines
ax.coastlines()

# Add gridlines and lat/lon labels
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.7, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {"size": 10}
gl.ylabel_style = {"size": 10}

# Place legend outside plot
ax.legend(
    loc='center right',
    bbox_to_anchor=(0.9, 0.5),
    frameon=False
)

# Title and save
ax.set_title("Time-Mean Flow of Total Current (1/1/24 - 5/29/25)")
plt.tight_layout()
plt.savefig("timemean.png", dpi=100)

# Animation of speed with normalized magnitudes

In [ ]:
def flow_direction_and_speed(ds, fig, tt, *args, **kwargs):
    da = ds.isel(time=tt)
    step = 3
    data_density = kwargs['plot_args']  # unpack from args
    normalized_u = da.u[::step, ::step] / da.speed
    normalized_v = da.v[::step, ::step] / da.speed
    
    projection = ccrs.PlateCarree()
    # Use the passed-in figure; create an axis on it:
    ax = fig.add_subplot(1, 1, 1, projection=projection)

    # Mask NaNs correctly
    frame = da.speed  # plotting handles NaNs well
    
    cmap = plt.get_cmap("jet").copy()
    cmap.set_bad("white")

    # Plot speed
    frame.plot(
        ax=ax,
        transform=projection,
        cmap=cmap,
        vmin=0,
        vmax=0.8,
        add_colorbar=True,
        cbar_kwargs={
            "label": "Flow Speed ($m\,s^{-1}$)",
        }
    )

    # Flow vectors
    ax.quiver(
        normalized_u.lon, normalized_u.lat,
        normalized_u, normalized_v,
        color='red',
        scale=30
    )

    #ax.set_extent([-127, -116, 32, 50], crs=projection)
    ax.set_extent([-124.5, -121.5, 36, 39], crs=projection)
    ax.coastlines()

    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.7, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {"size": 10}
    gl.ylabel_style = {"size": 10}

    suptitle = kwargs.get("title", "default title")
    fig.suptitle(suptitle)

    return fig, ax


In [ ]:
mov = Movie(ds_6km.chunk({'time':1}).isel(time=slice(-1000, None, 1)), 
            flow_direction_and_speed,
            input_check=False, dpi=100, 
            pixelwidth=1600,
            pixelheight=1240,
            plot_args=(normalized_u, normalized_v, data_density),  # <-- pass here
            title="California Coastal Flow (1/1/24 - 5/29/25)")

with ProgressBar():
    mov.save('speed-direction.mp4',
            progress=True,
            parallel=True,
            overwrite_existing=True,
            #parallel_compute_kwargs=dict(scheduler="processes", num_workers=6)
            )

# Subsetted around Bay area

In [ ]:
ds_6km_subset = ds_6km.where(
    (ds_6km.lat >= 36) & (ds_6km.lat <= 39) &
    (ds_6km.lon >= -124.5) & (ds_6km.lon <= -121.5),
    drop=True
)

In [ ]:
valid_counts_subset = ds_6km_subset.speed.notnull().sum(dim="time")
total_steps_subset = ds_6km_subset.dims["time"]
data_density_subset = (valid_counts_subset / total_steps_subset) * 100

normalized_u_subset = ds_6km_subset.u / ds_6km_subset.speed
normalized_v_subset = ds_6km_subset.v / ds_6km_subset.speed

In [ ]:
mov = Movie(ds_6km_subset.chunk({'time':1}).isel(time=slice(-1000, None, 1)), 
            flow_direction_and_speed,
            input_check=False, dpi=100, 
            pixelwidth=1600,
            pixelheight=1240,
            plot_args=(normalized_u_subset, normalized_v_subset, data_density_subset),  # <-- pass here
            title="Bay Area Coastal Flow (1/1/24 - 5/29/25)")

with ProgressBar():
    mov.save('bay-area-speed-direction.mp4',
            progress=True,
            parallel=True,
            overwrite_existing=True,
            #parallel_compute_kwargs=dict(scheduler="processes", num_workers=6)
            )

# Rotary power spectra

In [54]:
import importlib
import hf_scripts as hf

In [ ]:
importlib.reload(hf)

In [55]:
# First, let's shrink our time selection to April 2025.

cali_6km_april2025 = ds_6km.sel(time=slice('2025-04-01', '2025-04-30'))

We want to ensure that our data is fully complete, so we will focus on April 2025:

In [56]:
cali_6km_april2025_density = hf.time_density(cali_6km_april2025, var="speed", dim="time")

/home/jovyan/hf/hf_scripts.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  total_steps = ds.dims[dim]


In [231]:
from matplotlib.patches import Rectangle
import cartopy.feature as cfeature

mask = cali_6km_april2025_density.where(cali_6km_april2025_density == 100)

plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add coastlines for context
ax.coastlines()

ax.set_extent([-126, -116, 32, 47.5])

# Plot the mask: True points as 1, False as NaN (so only full data points show)
mask.plot.pcolormesh(
    ax=ax,
    cmap='Greens',      # Choose color, e.g. greens for coverage
    add_colorbar=False,
)

gl = ax.gridlines(draw_labels=True, linestyle='--', color='gray', alpha=0.5)
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

# Define box bounds (lon_min, lat_min, width, height)
boxes = {
    "box1": [-124.6, 44.1, 0.35, 0.5],
    "box2": [-123.75, 37.9, 0.62, 0.5],
    "box3": [-123.35, 37.3, 0.7, 0.5],
    "box4": [-122.85, 36.4, 0.8, 0.45],
    "box5": [-121.5, 34.5, 0.8, 0.6], #lon_min, lat_min, width, height (degrees)
}

# Add rectangle patch (highlighted box)
for key in boxes:
    lon_min, lat_min, width, height = boxes[key]  # unpack the box
    rect = Rectangle(
        (lon_min, lat_min),
        width,
        height,
        linewidth=2,
        #edgecolor='red',
        facecolor='red',
        alpha=0.5,
        transform=ccrs.PlateCarree()  # Important! Matches the map projection
    )
    ax.add_patch(rect)

plt.title("Map of Points with Full Data Coverage (April 2025)")
plt.savefig("completedata.png")

In [276]:
subsets = {}
legend_labels = {}

for key in boxes:
    lon_min, lat_min, width, height = boxes[key]  # unpack the box
    subsets[key] = cali_6km_april2025.sel(
        lon=slice(lon_min, lon_min + width), 
        lat=slice(lat_min, lat_min + height)
    )
    legend_labels[key] = f"{lat_min} - {lat_min + height}"
    
z_aprils = {}

for box in subsets:
    z_aprils[box] = subsets[box].u + 1j * subsets[box].v
    z_aprils[box] = z_aprils[box].drop_vars(['time_run'])
    z_aprils[box] = z_aprils[box].mean(dim=["lat", "lon"])

In [60]:
rotary_psd = xrft.power_spectrum(
    z_april,
    dim='time',
    detrend='constant',      # or 'linear'
    window=True,             # Apply Hanning window
    scaling='density',       # Return power spectral density
    true_phase=True,         # Keep complex output for rotary separation
    shift=False              # Don't shift zero-freq to center
)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xrft/xrft.py:47: FutureWarning: Please provide the name of window adhering to scipy.signal.windows. The boolean option will be deprecated in future releases.
  warnings.warn(


In [165]:
# Spatial average over lat and lon
avg_psd = rotary_psd.mean(dim=["lat", "lon"])

dt_days = 24 # hours

# Get frequency and PSD values
f = avg_psd["freq_time"].values     # frequency axis (includes negative and positive)
S = avg_psd.values             # PSD values (same shape as f)

# Convert frequency to cycles per day
f_cpd = f / (1 / dt_days)

# Plot
fig, ax = plt.subplots(figsize=(8, 4))

# Plot negative frequencies to the left (Clockwise)
ax.semilogy(f_cpd[f_cpd < 0], S[f < 0], label="Clockwise (CW)", color="blue", linestyle="-", linewidth=1.5)

# Plot positive frequencies to the right (Counterclockwise)
ax.semilogy(f_cpd[f_cpd > 0], S[f > 0], label="Counterclockwise (CCW)", color="blue", linestyle="-", linewidth=1.5)

# Labels and grid
ax.set_xlabel("Frequency (cpd)")
ax.set_ylabel("Power Spectral Density")
ax.set_title("Rotary PSD (CW on left, CCW on right)")
ax.grid(True, which="both", linestyle=":")
ax.axvline(0, color="gray", linewidth=0.8, linestyle=":")

# Legend and formatting
ax.legend()
plt.tight_layout()
plt.savefig("avg_rotary_psd.png", dpi=150)
plt.show()

In [249]:
import scipy.signal as sg  #Package for signal analysis
import scipy.ndimage as si #Another package for signal analysis

from multitaper import MTSpec  #using German Prieto's multitaper package, https://github.com/gaprieto/multitaper
from scipy import fft as spfft
from scipy.fft import fft 
from scipy.stats import chi2
from scipy.special import digamma

from matplotlib.cm import get_cmap


NOTE THAT THE HIGHEST LATITUDE POWER SPECTRUM IS NOT SCALED.

In [293]:
dt = 1/24
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

cmap = plt.get_cmap("tab10")
colors = list(cmap.colors)

handles = []
labels = []

for i, key in enumerate(z_aprils):
    box = z_aprils[key]
    fo, So = sg.periodogram(box.values-np.mean(box.values), fs=1/dt, return_onesided=False) #fs = sampling frequency (cyclic)
    
    P = 3
    spec = MTSpec(box.values-np.mean(box.values), nw=P, dt=dt, iadapt=1, nfft=len(box.values))
    S = np.ravel(spec.spec)
    f = np.ravel(spec.freq)
    
    print((f[1]-f[0])*np.sum(S)) #verify variance is approximately recovered

    color = colors[i % len(colors)]

    offset = 10**-i  # Shift each spectrum up by 10×
    
    h1, = ax.semilogy( f[f>0],S[f>0] * offset,linewidth=1.5,color=color)#plot positive side
    ax.semilogy( f[f<0],S[f<0] * offset,linewidth=1.5,color=color)#plot negative side
    ax.autoscale(enable=True, tight=False)
    ax.set_xlim(-3, 3)
    ax.set_ylim(1e-10, 1e0)
    #fig.tight_layout()

    label = legend_labels.get(key, key)  # fallback to key if missing
    handles.append(h1)
    labels.append(label)

#show smoothing extents at different frequencies
#io=(1,50,200,800,1600)
#yo=(10**2,10**1,10**0,10**(-0.5),10**(-0.75))
#for i in range(np.size(io)):
#    if i==0:
#        plt.hlines(yo[i],f[io[i]],f[io[i]+P],linewidth=3,color='k')
#    else:
#        plt.hlines(yo[i],f[io[i]-P],f[io[i]+P],linewidth=3,color='k')

#ax.vlines(tidefreq()/2/np.pi,ax.get_ylim()[0],ax.get_ylim()[1],linestyle=":", color="black")
#ax.vlines(corfreq(ds["lat"].data)/2/np.pi,ax.get_ylim()[0],ax.get_ylim()[1],linestyle="--", color="gray",linewidth=2)

ax.legend(handles, labels, title='Lat. Range (°N)', loc='upper right', fontsize='small', bbox_to_anchor=(1.2, 1))
plt.xlabel('clockwise <- Frequency (cycles/day) -> anti-clockwise')
plt.ylabel('Power Spectral Density (cm$^2$/s$^2$ days)')
plt.title(f'Multitaper Rotary Spectral Estimate with P={P}');
plt.tight_layout()
plt.savefig("multitaper.png")
plt.close()

0.03605037927627563
0.013916837982833387
0.013469865545630458
0.009193827398121357
0.02007131278514862


In [245]:
z_aprils['box1'].values

array([ 0.0334    +2.89599985e-01j,  0.0146    +3.07999998e-01j,
        0.0086    +3.15199971e-01j,  0.0222    +3.32599998e-01j,
        0.04799999+3.46199989e-01j,  0.0616    +3.55799943e-01j,
        0.065     +3.25799972e-01j,  0.07039999+2.97600001e-01j,
        0.0444    +2.99199969e-01j,  0.0062    +2.89199978e-01j,
       -0.0304    +2.39199981e-01j, -0.07179999+1.84799999e-01j,
       -0.09059999+1.32200003e-01j, -0.1098    +1.05399996e-01j,
       -0.12640001+5.61999977e-02j, -0.12580001+5.49999960e-02j,
       -0.11480001+5.35999984e-02j, -0.095     +8.69999975e-02j,
       -0.067     +1.13999993e-01j, -0.04820001+1.09799996e-01j,
       -0.05      +1.01599999e-01j, -0.06739999+5.88000007e-02j,
       -0.0846    +4.29999977e-02j, -0.1004    +1.20000001e-02j,
       -0.09720001-2.13999990e-02j, -0.1006    -3.18000019e-02j,
       -0.0876    -4.07999940e-02j, -0.08079999-4.39999998e-02j,
       -0.0678    +8.40000063e-03j, -0.0496    -2.59999861e-03j,
       -0.0348    -4.3399